In [ ]:
import itertools as it
import operator
import re
import subprocess
import tempfile
import time
from collections import Counter
from functools import partial
from pathlib import Path

import awkward as ak
import bottleneck as bn
import duckdb
import gfapy
import holoviews as hv
import ibis
import matplotlib.pyplot as plt
import numba
import numpy as np
import pandas as pd
import parasail
import polars as pl
import pyabpoa
import pyarrow as pa
import pyarrow.compute as pc
import pyarrow.dataset as ds
import pyfastx
import pysam
import spoa
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from pyarrow import csv
from pywfa import WavefrontAligner
from tqdm.auto import tqdm, trange

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.sequencing.align as align
import paulssonlab.sequencing.consensus as con
import paulssonlab.sequencing.gfa as sgfa
import paulssonlab.sequencing.io as sio
import paulssonlab.sequencing.processing as processing
from paulssonlab.util.sequence import reverse_complement

In [ ]:
hv.extension("bokeh")

In [ ]:
%load_ext pyinstrument
import line_profiler
import pyinstrument

%load_ext line_profiler

In [ ]:
pl.enable_string_cache()

# Config

In [ ]:
gfa_filename = "/home/jqs1/scratch/jqs1/sequencing/230930_alignment_test/230707_repressilators/pLIB419.gfa"

In [ ]:
gfa = gfapy.Gfa.from_file(gfa_filename)

# Identify variants

# Realign

In [ ]:
df = pl.read_ipc(
    "/home/jqs1/scratch/jqs1/sequencing/230930_alignment_test/230707_repressilators/dorado_0.4.0/uncompressed/prepared/consensus_spoa2/align2/combined.arrow"
)

In [ ]:
df.estimated_size(unit="mb")

In [ ]:
{col: df.get_column(col).estimated_size(unit="mb") for col in df.columns}

In [ ]:
path = df[0, "path_consensus"].to_list()
consensus_seq = df[0, "consensus_seq"]
cigar = df[0, "cg"]

In [ ]:
ref_seq = sgfa.assemble_seq_from_path(gfa, path)

In [ ]:
score, cigar = align.pairwise_align(
    consensus_seq,
    ref_seq,
    degenerate=True,
    gap_opening=12,
    gap_extension=3,
    match=1,
    mismatch=0,
    parasail_algorithm="nw",
)
(score, cigar)